In [2]:
import time
import os
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import re

# 抓取單個earnings call transcripts

In [13]:
# 初始化 WebDriver
driver = webdriver.Chrome()
base_url = "https://www.fool.com/earnings-call-transcripts/"
driver.get(base_url)

# 等待並點擊「I Accept」按鈕
try:
    accept_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))
    )
    accept_button.click()
    print("Clicked 'I Accept' button.")
except Exception as e:
    print(f"Error clicking 'I Accept' button: {e}")

# 點擊 "Load More" 按鈕三次 按一次會load 60筆
# for i in range(3):
#     try:
#         load_more_button = WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.XPATH, "//span[contains(text(),'Load More')]")))
#         driver.execute_script("arguments[0].click();", load_more_button)
#         print(f"Clicked 'Load More' button {i+1} time(s).")
#         time.sleep(3)  # 等待页面加载
#     except Exception as e:
#         print("Error:", e)
#         break    

# 已處理過url
processed_urls = set()
transcript_links = driver.find_elements(By.CSS_SELECTOR, ".text-gray-1100 a")
link = transcript_links[3]
transcript_url = link.get_attribute("href")
print('transcript_url: ', transcript_url)

try:
    transcript_response = requests.get(transcript_url)
    transcript_soup = BeautifulSoup(transcript_response.text, "html.parser")
    
    # 抓取 header 標籤中的 h1 標籤內容
    headers = transcript_soup.find_all('header')
    if len(headers) > 1:
        header_h1 = headers[1].find('h1')
        if header_h1:
            header_text = header_h1.get_text(strip=True)
            
            # 抓取股票代號 stock_symbol
            match = re.search(r'\((.*?)\)', header_text)
            if match:
                stock_symbol = match.group(1)
                print('stock_symbol:',stock_symbol)
            else:
                print("沒有找到括號內的文字")
                
        else:
            header_text = "No h1 found in the second header"
    else:
        header_text = "Less than 2 headers found"
    print('header_text:', header_text)
    
    # 抓取 <span id="date"> 和 <em id="time"> 標籤內容
    date_span = transcript_soup.find('span', id='date')
    time_em = transcript_soup.find('em', id='time')
    date_text = date_span.get_text(strip=True) if date_span else "No date found"
    time_text = time_em.get_text(strip=True) if time_em else "No time found"
    
    print('date_text:', date_text)
    print('time_text:', time_text)
    print('我是分隔線-------------------------------------')

    # 抓取所有 <p> 標籤內容
    paragraphs = transcript_soup.find_all('p')
    transcript_text = "\n\n".join(paragraph.get_text(strip=True) for paragraph in paragraphs)

    # 尋找 "Image source: The Motley Fool. 並取之後的文字
    truncate_front_index = transcript_text.find("Image source: The Motley Fool.")
    if truncate_front_index != -1:
        transcript_text = transcript_text[truncate_front_index + len("Image source: The Motley Fool.") + 80:]
    # 尋找 "Duration: 並取之前的文字
    truncate_back_index = transcript_text.find("Duration:")
    if truncate_back_index != -1:
        transcript_text = transcript_text[:truncate_back_index]
    print('transcript_text', transcript_text)
except Exception as e:
    print(f"Error processing URL {len(processed_urls)}: {e}")
print('我是分隔線-------------------------------------')

# 關閉 WebDriver
driver.quit()

Clicked 'I Accept' button.
transcript_url:  https://www.fool.com/earnings/call-transcripts/2024/05/20/palo-alto-networks-panw-q3-2024-earnings-call-tran/
stock_symbol: PANW
header_text: Palo Alto Networks (PANW) Q3 2024 Earnings Call Transcript
date_text: May 20, 2024
time_text: 4:30 p.m. ET
我是分隔線-------------------------------------
transcript_text alter Pritchard

Good day, everyone, and welcome to Palo Alto Networks' fiscal third quarter 2024 earnings conference call. I'm Walter Pritchard, senior vice president of investor relations and corporate development. Please note that this call is being recorded today, Monday, May 20th, 2024, at 1:30 p.m. Pacific Time.

With me on today's call to discuss second quarter results are Nikesh Arora, our chairman and chief executive officer; and Dipak Golechha, our chief financial officer. Following our prepared remarks, Lee Klarich, our chief product officer, will join us for the question-and-answer portion. You can find the press release and oth

# 抓取多筆資料

In [ ]:
%%time
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# 初始化 WebDriver
driver = webdriver.Chrome()
base_url = "https://www.fool.com/earnings-call-transcripts/"
driver.get(base_url)

# 等待並點擊「I Accept」按鈕
try:
    accept_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))
    )
    accept_button.click()
    print("Clicked 'I Accept' button.")
except Exception as e:
    print(f"Error clicking 'I Accept' button: {e}")

#點擊 "Load More" 按鈕三次，每次會加載20筆
for i in range(400):
    try:
        load_more_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//span[contains(text(),'Load More')]")))
        driver.execute_script("arguments[0].click();", load_more_button)
        print(f"Clicked 'Load More' button {i+1} time(s).")
        time.sleep(3)  # 等待页面加载
    except Exception as e:
        print("Error:", e)
        break    

# 已處理過的 URL 集合
processed_urls = set()
transcript_links = driver.find_elements(By.CSS_SELECTOR, ".text-gray-1100 a")

# 定義欄位名稱
columns = ['stock_symbol', 'title', 'date', 'paragraphs']

# 建立一個空的 DataFrame，並設定欄位名稱
df = pd.DataFrame(columns=columns)

# 迴圈遍歷每個 transcript link
for index, link in enumerate(transcript_links):
    transcript_url = link.get_attribute("href")

    # 檢查 URL 是否包含 "author" 或已處理過
    if "author" in transcript_url:
        print(f"Skipping author URL {index + 1}/{len(transcript_links)}: {transcript_url}")
        continue

    if transcript_url in processed_urls:
        print(f"Skipping duplicate URL {index + 1}/{len(transcript_links)}: {transcript_url}")
        continue

    print(f'Processing transcript URL {index + 1}/{len(transcript_links)}: {transcript_url}')
    processed_urls.add(transcript_url)

    try:
        transcript_response = requests.get(transcript_url)
        transcript_soup = BeautifulSoup(transcript_response.text, "html.parser")
        
        # 抓取 header 標籤中的 h1 標籤內容
        headers = transcript_soup.find_all('header')
        if len(headers) > 1:
            header_h1 = headers[1].find('h1')
            if header_h1:
                header_text = header_h1.get_text(strip=True)
                
                # 抓取股票代號 stock_symbol
                match = re.search(r'\((.*?)\)', header_text)
                if match:
                    stock_symbol = match.group(1)
                else:
                    stock_symbol = "No stock symbol found"
            else:
                header_text = "No h1 found in the second header"
                stock_symbol = "No stock symbol found"
        else:
            header_text = "Less than 2 headers found"
            stock_symbol = "No stock symbol found"
        
        # 抓取 <span id="date"> 和 <em id="time"> 標籤內容
        date_span = transcript_soup.find('span', id='date')
        time_em = transcript_soup.find('em', id='time')
        date_text = date_span.get_text(strip=True) if date_span else "No date found"
        time_text = time_em.get_text(strip=True) if time_em else "No time found"
        
        # 抓取所有 <p> 標籤內容
        paragraphs = transcript_soup.find_all('p')
        transcript_text = "\n\n".join(paragraph.get_text(strip=True) for paragraph in paragraphs)

        # 尋找 "Image source: The Motley Fool. 並取之後的文字
        truncate_front_index = transcript_text.find("Image source: The Motley Fool.")
        if truncate_front_index != -1:
            transcript_text = transcript_text[truncate_front_index + len("Image source: The Motley Fool.") + 80:]
        # 尋找 "Duration: 並取之前的文字
        truncate_back_index = transcript_text.find("Duration:")
        if truncate_back_index != -1:
            transcript_text = transcript_text[:truncate_back_index]
        
        # 將新的一行加入到 DataFrame
        new_row = pd.DataFrame([{
            'stock_symbol': stock_symbol,
            'title': header_text,
            'date': date_text,
            'paragraphs': transcript_text
        }])
        df = pd.concat([df, new_row], ignore_index=True)
        
    except Exception as e:
        print(f"Error processing URL {index + 1}: {e}")

Clicked 'I Accept' button.
Clicked 'Load More' button 1 time(s).
Clicked 'Load More' button 2 time(s).
Clicked 'Load More' button 3 time(s).
Clicked 'Load More' button 4 time(s).
Clicked 'Load More' button 5 time(s).
Clicked 'Load More' button 6 time(s).
Clicked 'Load More' button 7 time(s).
Clicked 'Load More' button 8 time(s).
Clicked 'Load More' button 9 time(s).
Clicked 'Load More' button 10 time(s).
Clicked 'Load More' button 11 time(s).
Clicked 'Load More' button 12 time(s).
Clicked 'Load More' button 13 time(s).
Clicked 'Load More' button 14 time(s).
Clicked 'Load More' button 15 time(s).
Clicked 'Load More' button 16 time(s).
Clicked 'Load More' button 17 time(s).
Clicked 'Load More' button 18 time(s).
Clicked 'Load More' button 19 time(s).
Clicked 'Load More' button 20 time(s).
Clicked 'Load More' button 21 time(s).
Clicked 'Load More' button 22 time(s).
Clicked 'Load More' button 23 time(s).
Clicked 'Load More' button 24 time(s).
Clicked 'Load More' button 25 time(s).
Clicked

Skipping author URL 344/3060: https://www.fool.com/author/20032/
Skipping duplicate URL 345/3060: https://www.fool.com/earnings/call-transcripts/2024/05/08/turtle-beach-hear-q1-2024-earnings-call-transcript/
Processing transcript URL 346/3060: https://www.fool.com/earnings/call-transcripts/2024/05/08/pubmatic-pubm-q1-2024-earnings-call-transcript/
Skipping author URL 347/3060: https://www.fool.com/author/20032/
Skipping duplicate URL 348/3060: https://www.fool.com/earnings/call-transcripts/2024/05/08/pubmatic-pubm-q1-2024-earnings-call-transcript/
Processing transcript URL 349/3060: https://www.fool.com/earnings/call-transcripts/2024/05/07/lyft-lyft-q1-2024-earnings-call-transcript/
Skipping author URL 350/3060: https://www.fool.com/author/20032/
Skipping duplicate URL 351/3060: https://www.fool.com/earnings/call-transcripts/2024/05/07/lyft-lyft-q1-2024-earnings-call-transcript/
Processing transcript URL 352/3060: https://www.fool.com/earnings/call-transcripts/2024/05/07/sonos-sono-q2-

Skipping author URL 416/3060: https://www.fool.com/author/20032/
Skipping duplicate URL 417/3060: https://www.fool.com/earnings/call-transcripts/2024/05/07/coherent-cohr-q3-2024-earnings-call-transcript/
Processing transcript URL 418/3060: https://www.fool.com/earnings/call-transcripts/2024/05/07/crocs-crox-q1-2024-earnings-call-transcript/
Skipping author URL 419/3060: https://www.fool.com/author/20032/
Skipping duplicate URL 420/3060: https://www.fool.com/earnings/call-transcripts/2024/05/07/crocs-crox-q1-2024-earnings-call-transcript/
Processing transcript URL 421/3060: https://www.fool.com/earnings/call-transcripts/2024/05/07/walt-disney-dis-q2-2024-earnings-call-transcript/
Skipping author URL 422/3060: https://www.fool.com/author/20032/
Skipping duplicate URL 423/3060: https://www.fool.com/earnings/call-transcripts/2024/05/07/walt-disney-dis-q2-2024-earnings-call-transcript/
Processing transcript URL 424/3060: https://www.fool.com/earnings/call-transcripts/2024/05/07/datadog-ddog

Skipping author URL 488/3060: https://www.fool.com/author/20032/
Skipping duplicate URL 489/3060: https://www.fool.com/earnings/call-transcripts/2024/05/03/fubotv-fubo-q1-2024-earnings-call-transcript/
Processing transcript URL 490/3060: https://www.fool.com/earnings/call-transcripts/2024/05/03/xpo-xpo-q1-2024-earnings-call-transcript/
Skipping author URL 491/3060: https://www.fool.com/author/20032/
Skipping duplicate URL 492/3060: https://www.fool.com/earnings/call-transcripts/2024/05/03/xpo-xpo-q1-2024-earnings-call-transcript/
Processing transcript URL 493/3060: https://www.fool.com/earnings/call-transcripts/2024/05/03/brookfield-renewable-partners-bep-q1-2024-earnings/
Skipping author URL 494/3060: https://www.fool.com/author/20032/
Skipping duplicate URL 495/3060: https://www.fool.com/earnings/call-transcripts/2024/05/03/brookfield-renewable-partners-bep-q1-2024-earnings/
Processing transcript URL 496/3060: https://www.fool.com/earnings/call-transcripts/2024/05/03/apa-apa-q1-2024-

Skipping author URL 560/3060: https://www.fool.com/author/20032/
Skipping duplicate URL 561/3060: https://www.fool.com/earnings/call-transcripts/2024/05/02/booking-holdings-bkng-q1-2024-earnings-call-transc/
Processing transcript URL 562/3060: https://www.fool.com/earnings/call-transcripts/2024/05/02/digital-realty-trust-dlr-q1-2024-earnings-call-tra/
Skipping author URL 563/3060: https://www.fool.com/author/20032/
Skipping duplicate URL 564/3060: https://www.fool.com/earnings/call-transcripts/2024/05/02/digital-realty-trust-dlr-q1-2024-earnings-call-tra/
Processing transcript URL 565/3060: https://www.fool.com/earnings/call-transcripts/2024/05/02/inmode-inmd-q1-2024-earnings-call-transcript/
Skipping author URL 566/3060: https://www.fool.com/author/20032/
Skipping duplicate URL 567/3060: https://www.fool.com/earnings/call-transcripts/2024/05/02/inmode-inmd-q1-2024-earnings-call-transcript/
Processing transcript URL 568/3060: https://www.fool.com/earnings/call-transcripts/2024/05/02/bl

...

Skipping author URL 2732/3060: https://www.fool.com/author/20032/
Skipping duplicate URL 2733/3060: https://www.fool.com/earnings/call-transcripts/2023/12/05/joann-joan-q3-2024-earnings-call-transcript/
Processing transcript URL 2734/3060: https://www.fool.com/earnings/call-transcripts/2023/12/01/domo-domo-q3-2024-earnings-call-transcript/
Skipping author URL 2735/3060: https://www.fool.com/author/20032/
Skipping duplicate URL 2736/3060: https://www.fool.com/earnings/call-transcripts/2023/12/01/domo-domo-q3-2024-earnings-call-transcript/
Processing transcript URL 2737/3060: https://www.fool.com/earnings/call-transcripts/2023/12/01/uipath-path-q3-2024-earnings-call-transcript/
Skipping author URL 2738/3060: https://www.fool.com/author/20032/
Skipping duplicate URL 2739/3060: https://www.fool.com/earnings/call-transcripts/2023/12/01/uipath-path-q3-2024-earnings-call-transcript/
Processing transcript URL 2740/3060: https://www.fool.com/earnings/call-transcripts/2023/11/30/elastic-estc-q2-

Skipping author URL 2804/3060: https://www.fool.com/author/20032/
Skipping duplicate URL 2805/3060: https://www.fool.com/earnings/call-transcripts/2023/11/22/tremor-international-trmr-q3-2023-earnings-call-tr/
Processing transcript URL 2806/3060: https://www.fool.com/earnings/call-transcripts/2023/11/22/baozun-bzun-q3-2023-earnings-call-transcript/
Skipping author URL 2807/3060: https://www.fool.com/author/20032/
Skipping duplicate URL 2808/3060: https://www.fool.com/earnings/call-transcripts/2023/11/22/baozun-bzun-q3-2023-earnings-call-transcript/
Processing transcript URL 2809/3060: https://www.fool.com/earnings/call-transcripts/2023/11/22/nordstrom-jwn-q3-2023-earnings-call-transcript/
Skipping author URL 2810/3060: https://www.fool.com/author/20032/
Skipping duplicate URL 2811/3060: https://www.fool.com/earnings/call-transcripts/2023/11/22/nordstrom-jwn-q3-2023-earnings-call-transcript/
Processing transcript URL 2812/3060: https://www.fool.com/earnings/call-transcripts/2023/11/22/a

Skipping author URL 2876/3060: https://www.fool.com/author/20032/
Skipping duplicate URL 2877/3060: https://www.fool.com/earnings/call-transcripts/2023/11/15/sonos-sono-q4-2023-earnings-call-transcript/
Processing transcript URL 2878/3060: https://www.fool.com/earnings/call-transcripts/2023/11/15/cisco-systems-csco-q1-2024-earnings-call-transcrip/
Skipping author URL 2879/3060: https://www.fool.com/author/20032/
Skipping duplicate URL 2880/3060: https://www.fool.com/earnings/call-transcripts/2023/11/15/cisco-systems-csco-q1-2024-earnings-call-transcrip/
Processing transcript URL 2881/3060: https://www.fool.com/earnings/call-transcripts/2023/11/15/palo-alto-networks-panw-q1-2024-earnings-call-tran/
Skipping author URL 2882/3060: https://www.fool.com/author/20032/
Skipping duplicate URL 2883/3060: https://www.fool.com/earnings/call-transcripts/2023/11/15/palo-alto-networks-panw-q1-2024-earnings-call-tran/
Processing transcript URL 2884/3060: https://www.fool.com/earnings/call-transcripts

Skipping author URL 2948/3060: https://www.fool.com/author/20032/
Skipping duplicate URL 2949/3060: https://www.fool.com/earnings/call-transcripts/2023/11/09/canopy-growth-cgc-q2-2024-earnings-call-transcript/
Processing transcript URL 2950/3060: https://www.fool.com/earnings/call-transcripts/2023/11/09/plug-power-plug-q3-2023-earnings-call-transcript/
Skipping author URL 2951/3060: https://www.fool.com/author/20032/
Skipping duplicate URL 2952/3060: https://www.fool.com/earnings/call-transcripts/2023/11/09/plug-power-plug-q3-2023-earnings-call-transcript/
Processing transcript URL 2953/3060: https://www.fool.com/earnings/call-transcripts/2023/11/09/doubleverify-dv-q3-2023-earnings-call-transcript/
Skipping author URL 2954/3060: https://www.fool.com/author/20032/
Skipping duplicate URL 2955/3060: https://www.fool.com/earnings/call-transcripts/2023/11/09/doubleverify-dv-q3-2023-earnings-call-transcript/
Processing transcript URL 2956/3060: https://www.fool.com/earnings/call-transcripts/

Skipping author URL 3020/3060: https://www.fool.com/author/20032/
Skipping duplicate URL 3021/3060: https://www.fool.com/earnings/call-transcripts/2023/11/09/invitae-nvta-q3-2023-earnings-call-transcript/
Processing transcript URL 3022/3060: https://www.fool.com/earnings/call-transcripts/2023/11/09/mgm-resorts-international-mgm-q3-2023-earnings-cal/
Skipping author URL 3023/3060: https://www.fool.com/author/20032/
Skipping duplicate URL 3024/3060: https://www.fool.com/earnings/call-transcripts/2023/11/09/mgm-resorts-international-mgm-q3-2023-earnings-cal/
Processing transcript URL 3025/3060: https://www.fool.com/earnings/call-transcripts/2023/11/09/infinera-infn-q3-2023-earnings-call-transcript/
Skipping author URL 3026/3060: https://www.fool.com/author/20032/
Skipping duplicate URL 3027/3060: https://www.fool.com/earnings/call-transcripts/2023/11/09/infinera-infn-q3-2023-earnings-call-transcript/
Processing transcript URL 3028/3060: https://www.fool.com/earnings/call-transcripts/2023/

In [12]:
df

,stock_symbol,title,date,paragraphs
0,XPEV,XPeng (XPEV) Q1 2024 Earnings Call Transcript,"May 21, 2024","o, ladies and gentlemen. Thank you for standin..."
1,LOW,Lowe's Companies (LOW) Q1 2024 Earnings Call T...,"May 21, 2024","rator\n\nGood morning, everyone. Welcome to Lo..."
2,M,Macy's (M) Q1 2024 Earnings Call Transcript,"May 21, 2024","ngs, and welcome to the Macy's, Inc. first-qua..."
3,YMM,Full Truck Alliance (YMM) Q1 2024 Earnings Cal...,"May 21, 2024","Operator\n\nLadies and gentlemen, good day and..."
4,ZM,Zoom Video Communications (ZM) Q1 2025 Earning...,"May 20, 2024","ET\n\nKelcey McKinley\n\nHello, everyone, and..."
...,...,...,...,...
3913,UNM,Unum Group (UNM) Q2 2022 Earnings Call Transcript,"Aug 03, 2022","Hello, everyone, and welcome to the Unum Group..."
3914,ALE,ALLETE (ALE) Q2 2022 Earnings Call Transcript,"Aug 03, 2022","od day, and welcome to the ALLETE second quart..."
3915,CVS,CVS Health (CVS) Q2 2022 Earnings Call Transcript,"Aug 03, 2022","\nLadies and gentlemen, good morning, and welc..."
3916,LAMR,Lamar Advertising (LAMR) Q2 2022 Earnings Call...,"Aug 03, 2022","perator\n\nExcuse me, everyone. We now have Se..."


In [14]:
import pandas as pd
df = pd.read_csv('scraping_data/0531_no_label_data.csv')
df

,stock_symbol,title,date,paragraphs
0,COST,Costco Wholesale (COST) Q3 2024 Earnings Call ...,"May 30, 2024",rator\n\nThank you for standing by. My name is...
1,MRVL,Marvell Technology (MRVL) Q1 2025 Earnings Cal...,"May 30, 2024","perator\n\nGood afternoon, and welcome to Marv..."
2,GPS,Gap (GPS) Q1 2024 Earnings Call Transcript,"May 30, 2024","ternoon, ladies and gentlemen. I would like to..."
3,DG,Dollar General (DG) Q1 2024 Earnings Call Tran...,"May 30, 2024","tor\n\nGood morning. My name is Donna, and I w..."
4,DLTH,Duluth (DLTH) Q1 2024 Earnings Call Transcript,"May 30, 2024","od morning, everyone, and welcome to the Dulut..."
...,...,...,...,...
6373,AXTI,AXT (AXTI) Q4 2021 Earnings Call Transcript,"Feb 16, 2022","fternoon, everyone, and welcome to AXT's fourt..."
6374,BCOV,Brightcove (BCOV) Q4 2021 Earnings Call Transc...,"Feb 16, 2022",tings. Welcome to the Brightcove fourth quarte...
6375,OCDX,Ortho Clinical Diagnostics Holdings plc (OCDX)...,"Feb 16, 2022",",5:00 p.m. ET\n\nOperator\n\nWelcome to Ortho ..."
6376,APP,AppLovin Corporation (APP) Q4 2021 Earnings Ca...,"Feb 16, 2022",\nOperator\n\nGreetings. Welcome to AppLovin C...


In [17]:
# 嘗試將日期解析為 datetime，如果失敗則設為 NaT
import pandas as pd
from datetime import datetime

convert_date_df = df.copy()

# 自定義日期轉換函數
def convert_date(date_str):
    if isinstance(date_str, str):
        try:
            # 定義日期格式
            date_obj = datetime.strptime(date_str, "%b %d, %Y")
            # 轉換為所需格式
            return date_obj.strftime("%Y-%m-%d")
        except ValueError:
            # 返回 None 或其他值以表示轉換失敗
            return None
    else:
        # 返回 None 表示轉換失敗
        return None

# 假設 df 是之前的 DataFrame
# 轉換日期欄位
convert_date_df['date'] = convert_date_df['date'].apply(convert_date)

# 檢查是否有轉換失敗的日期
print(convert_date_df[convert_date_df['date'].isna()])

convert_date_df = convert_date_df.dropna(subset=['date'])

# 顯示處理後的 DataFrame
convert_date_df

               stock_symbol  \
1453                   SNDL   
1566                   ESGR   
1741                    STX   
1828                   KVUE   
2068                    AMC   
4114  No stock symbol found   
4790  No stock symbol found   

                                                  title  date  \
1453              SNDL (SNDL) QEarnings Call Transcript  None   
1566  Enstar Group (ESGR) Q2 2023 Earnings Call Tran...  None   
1741  Seagate Technology Plc (STX) QEarnings Call Tr...  None   
1828            Kenvue (KVUE) QEarnings Call Transcript  None   
2068  AMC Entertainment (AMC) QEarnings Call Transcript  None   
4114                          Less than 2 headers found  None   
4790                          Less than 2 headers found  None   

                                             paragraphs  
1453  L's second-quarter 2023 financial results conf...  
1566  Enstar's Second Quarter 2023 Earnings Audio Re...  
1741  welcome to the Seagate Technology fourth quart... 

,stock_symbol,title,date,paragraphs
0,COST,Costco Wholesale (COST) Q3 2024 Earnings Call ...,2024-05-30,rator\n\nThank you for standing by. My name is...
1,MRVL,Marvell Technology (MRVL) Q1 2025 Earnings Cal...,2024-05-30,"perator\n\nGood afternoon, and welcome to Marv..."
2,GPS,Gap (GPS) Q1 2024 Earnings Call Transcript,2024-05-30,"ternoon, ladies and gentlemen. I would like to..."
3,DG,Dollar General (DG) Q1 2024 Earnings Call Tran...,2024-05-30,"tor\n\nGood morning. My name is Donna, and I w..."
4,DLTH,Duluth (DLTH) Q1 2024 Earnings Call Transcript,2024-05-30,"od morning, everyone, and welcome to the Dulut..."
...,...,...,...,...
6373,AXTI,AXT (AXTI) Q4 2021 Earnings Call Transcript,2022-02-16,"fternoon, everyone, and welcome to AXT's fourt..."
6374,BCOV,Brightcove (BCOV) Q4 2021 Earnings Call Transc...,2022-02-16,tings. Welcome to the Brightcove fourth quarte...
6375,OCDX,Ortho Clinical Diagnostics Holdings plc (OCDX)...,2022-02-16,",5:00 p.m. ET\n\nOperator\n\nWelcome to Ortho ..."
6376,APP,AppLovin Corporation (APP) Q4 2021 Earnings Ca...,2022-02-16,\nOperator\n\nGreetings. Welcome to AppLovin C...


In [18]:
#convert_date_df.iloc[366]
convert_date_df.to_csv('0531_no_label_data.csv', index=False)